# Packaging Production Code

```{note}
Code for this section can be found in [`model-deployment/05-production/`](https://github.com/particle1331/model-deployment/tree/main/05-production).
```

Production code is designed to be deployed to end users as opposed to research code, which
is for experimentation, building proof of concepts, and research code tends to be more short term in nature. 
On the other hand, with production code, we have some new considerations:

- **Testability and maintainability** are huge.
We want to divide up our code into modules which are more extensible and easier to test.
We separate config from code where possible, and we ensure that functionality is tested and documented. We also look to ensure that our code adheres to standards like PEP 8 so that it's easy for others to
read.  

+++

- **Scalability and performance** are also important areas to consider.
With our production code, the code needs to be ready to be deployed to infrastructure that can be scaled. And in modern web applications, this typically means containerisation for vertical or horizontal scaling. And we'll be looking at that more in later sections.
Where appropriate, we might also refactor inefficient parts of the code base.  

+++

- Finally, we have to look at **reproducibility**.
The code resides under version control with clear processes for tracking releases and release versions, requirements, files, mark which dependencies and which versions are used by the code.

```{margin}
A **module** is basically just a Python file and a **package** is a
collection of modules.
```

So that's a quick overview of some of the key considerations with production code. In this article, we will be packaging up our machine learning model into what's called the Python package. But a package has certain standardized files which have to be present so that it can be published and then installed in other Python applications.
Packaging allows us to wrap our train model and make it available to other consuming applications as a dependency, but with the additional benefits of version control, clear metadata and reproducibility.

## Code overview

Now, in order to create a package, we'll have to follow certain Python standards and conventions and
we'll go into those in detail in this section.
We'll end up with a package structure that looks like this.
And by the time we get to the end of this section, all of this will be familiar.

```
.
├── regression_model
│   ├── config
│   ├── datasets
│   ├── processing
│   │   ├── __init__.py
│   │   ├── data_manager.py
│   │   ├── features.py
│   │   └── validation.py
│   ├── trained_models
│   ├── pipeline.py
│   ├── predict.py
│   ├── train_pipeline.py
│   ├── config.yml
│   └── VERSION
├── requirements
│   ├── requirements.txt
│   └── test_requirements.txt
├── tests
│   ├── conftest.py
│   ├── test_features.py
│   └── test_prediction.py
├── MANIFEST.in
├── mypy.ini
├── pyproject.toml
├── setup.py
└── tox.ini
```

Some of these files here the `MANIFEST.in`, `pyproject.toml`, `setup.py`, `mypy.ini` and `tox.ini` file are all things that we're using, either for packaging or for configuring things like linting and type checking. These are kinds of tooling configurations and will be coming back to discuss these in more detail below. We have a `requirements` directory, which is where we formalize the dependencies for our package and also for testing it. And we have a couple of sample tests in the `tests` directory.  


The `regression_model` directory is where the majority of our functionality is. In this directory, we have three key files `train_pipeline.py`, `predict.py` and `pipeline.py`.
These are sort of top level files for the key bits of functionality of the package. The `processing` directory contains different helper functions. We have the datasets that we need to train and test the models in the `datasets` directory. The `trained_models` directory is where we save the models that we're persisting
here as a pickle file so that it can be loaded in and accessed in the future. Finally, `config` contains the core configurations module which reads the `config.yml` file.


## Understanding the code structure

So basically, there are three ways that we broadly speaking use to determine how to create the structure for the production code.
Those three are conventions, packaging mandatory files and software engineering best practices, and we'll go into each of these and provide some examples.

### Conventions

So conventions are things that you don't have to do.
You're not going to get something like a Python syntax error if you don't do it.
But projects tend to adhere to some sort of convention in a particular area.
For example, the most obvious one is PEP 8, which is the Python style guide.
This provides loads of guidelines on how to write Python code.
Similar to that, we add a few linting tools into our project a bit later in the section.
Those will also be opinionated and want you to adhere to certain conventions.
And that can be really useful.


### Mandatory files

Next up, we have things that once we've made a certain decision, we then have to do so.
We've made the decision to package our model into a Python package once we've made that decision. Then, there are certain files that are required in order to do that, like the `setup.py` file and the `MANIFEST.in` file.
You have to have these files in order to publish a Python package, so certain decisions basically get made for us in terms of the structure and the files we need.
Again, broadly speaking, packaging is a really easy way for us to embed our model as a dependency in an application that runs.
There are other ways that you can persist and share your models.
But a Python package is a nice, easy way for us.
To do this makes it easy to version, makes it really easy to install into our consuming application.

### Best practices 

```{margin}
**End user functionality** 
```

Software engineering best practices can be separated into two broad categories. First, we have the stuff that is relevant for the end user of your package.
In our case, imagine that we published this package now as the end user, perhaps creating my application,
and I want to make predictions using this machine learning model package that I've installed.
In my application code, I'm going to import things like the ability to predict, right?
And that will come from the predict module. I don't want to also import all the training functions. In fact, that might be confusing for me.
So here we have a predict and a train separation and that feeds into just general software engineering good practices.

```{margin}
**Testing and maintainability** 
```

The second broad category relates to testing and maintanability.
For testing, you want to think about, can you write a single unit test for one specific bit
of functionality?
The most extreme case of this production code would be that we put almost everything in a single file, right?
Now, I think everybody intuitively knows that that would be messy and hard to maintain.
But you then continue that thread and keep following it.
And what you start to arrive at is this idea of separation of concerns, tests, ability.
For example, if we look at the `data_manager.py`,
we have a function for saving the pipeline and we have a function for removing old pipelines.
Now these could be one single function. But we choose not to have them as one function because now in a unit test, we can test removing old
pipelines as well as saving any pipeline. This makes testing a specific discrete unit of functionality a lot easier.

This leads to separation
of concerns, which leads to modularity.
And as the name implies, separation of concerns is about designing software systems so that different
areas of functionality are separated.
For example, this is why we have different modules for the `data_manager.py` module, which deals with loading the training data, and then we have the validation module in `validation.py`,
which deals with testing that our inputs to the model are correct.
Now you could easily have these in the same module, but we choose to separate them because they're doing different things. And again, that links to testability. This is related to the [SOLID principles](https://en.wikipedia.org/wiki/SOLID) where the first stands for single responsibility which again means that 
a class or module should have one job. Here we have `predict.py` only dealing with prediction and `train_pipeline.py` dealing with training models.
This is just engineering good practice that links to maintainability.

Imagine you want to modify this package, you're going to create a pull request.
And if you're the person reviewing this pull request and everything is all in one file, it's really difficult for you to track what's changed and understand exactly what you need to be reviewing. On the other hand, if the change is made in just a single small module, which only has code related to one area in it, it's way easier for you to immediately get the context, follow the flow of the
program and review that pull request.
So, this is the long term health of your code and making it nice and easy for maintainers.
You can apply similar thought processes when you're deciding how to structure your own projects. 

Another best practice related to maintainability is optimizing for readability. 
The [principle of least astonishment](https://en.wikipedia.org/wiki/Principle_of_least_astonishment) is one that is very useful.
You go into the code base, you look at the directories and the principle of least astonishment applies to a lot to naming. Do modules do what you expect them to do when you read the names and the same for classes and functions?
A good way to think about this is to imagine that the person who's going to maintain your code after you is a psychopath who knows where you live.


```{note}
Follow a standard where possible (PEP 8), pick a linting tool and then stick with it.
In our case, publishing models as packages gives us a nice structure to work with. This is a recommended starting point until you get to some more complex applications where publishing as a package may not be possible. At that point, it might be time to find a new standard.
Established conventions usually adhere to good practices, but not always.
And learning when an established convention actually isn't the right choice for you takes a lot of time and practice. But that's something to work towards.
```

## Package requirements

We have this `requirements` directory with two requirements files inside it.
Let's start with the `requirements.txt` file.
We have some comments at the top explaining some of the choices we've made here.
And then from line four onwards, starting at non-pay, we define the dependencies of our projects as
well as the version numbers of each of these dependencies which we were willing to accept, so which
is compatible with our particular package.

```
# We use compatible release functionality (see PEP 440 here: https://www.python.org/dev/peps/pep-0440/#compatible-release)
# to specify acceptable version ranges of our project dependencies. This gives us the flexibility to keep up with small
# updates/fixes, whilst ensuring we don't install a major update which could introduce backwards incompatible changes.
numpy
pandas>=1.2.0,<1.3.0
pydantic>=1.8.1,<1.9.0
scikit-learn>=0.24.0,<0.25.0
strictyaml>=1.3.2,<1.4.0
ruamel.yaml==0.16.12
feature-engine>=1.0.2,<1.1.0
joblib>=1.0.1,<1.1.0
```



Now, these versions all adhere to what's known as semantic versioning.
Semantic versioning looks like this.
You have a major version followed by a period followed by the minor version, followed by a period followed
by what's known as a patch version.
And for a well maintained package, you'd expect that a minor version increment does not break the API
and a major version increment is likely to break the API.
But you have to be a bit careful because some less well maintain packages may actually introduce breaking
changes, even in a minor version bump.
So what we've done in our requirements file is play it quite conservatively.
We're saying that anything for no greater than version one point to zero point zero, but it has to
be less than version one point two one.
So we're not allowing any minor version increases here.
And it's up to you and your projects, how much risk you want to take.
We can install all of these requirements with the commands, PIP install Fascher requirements.
60 likes and they will begin installing I'm going to cancel that for now and then we also have a test
requirements to file and this insects' at the top just means that we're going to install everything
in requirements as well as these.
So it's a way of just capturing everything in another requirements file.
And we've split the requirements into these two files because there will be scenarios where we don't
actually need to install these test requirements because these are only required when we want to test
our package or when we want to run style checks, lintang and type checks.
These two files are really important.
If we don't define which versions of our dependencies we expect, then it can result in our package
becoming very brittle and broken.
The most basic error in a package would be that you didn't have a requirement to file.
That would just mean that when somebody else tried to install and use the package, it would fail because
it wouldn't have its necessary dependencies.
A more likely scenario is that we neglect to include a version, and if there is no version specified,
PIP is just going to assume you want the latest version of a particular dependency.
And it may well be that that version has progressed and it's released new features, a new major version
perhaps, and it's got breaking changes in the API.
And that means that your package or our package in this case is going to break.
So it's really important for us to define these versions.
This requirements, texte approach to managing our projects dependencies is probably the most basic
way of doing dependency management in Python.
Nothing wrong with it at all.
Many of the biggest python open source projects out there use this exact approach.
However, just to avoid confusion.
If you're working in a project that uses a different dependency manager like, let's say, poetry or
Flit or Pippin's, then the the file used to define dependencies changes depending on which dependency
manager you're working with.
But the principle.
Of defining your dependencies and specifying the version ranges remains the same across all of the tools.
So focus on the principle, not so much on the exact syntax.